# Relational Database (PostgreSQL)

In [1]:
import psycopg2 as db

In [6]:
conn_string = "dbname='dataengineering' host='localhost' user='postgres' password='REPLACE_ME'"

In [7]:
conn = db.connect(conn_string)

In [8]:
# Allows Python code to execute PostgreSQL command in a database session.
cur = conn.cursor()

In [11]:
query = "insert into test1 values ('{name}', {id}, '{street}', '{city}', '{zip}')".format(name='Tom', 
                                                                                          id=1, 
                                                                                          street='Test St.',
                                                                                          city='Omaha',
                                                                                          zip='12345')

print(query)

insert into test1 values ('Tom', 1, 'Test St.', 'Omaha', '12345')


In [12]:
# Return a query string after arguments binding. The string returned is exactly the one that would be sent to the database running the execute() method or similar.
cur.mogrify(query)

b"insert into test1 values ('Tom', 1, 'Test St.', 'Omaha', '12345')"

In [15]:
# Another way to write query
query2 = "insert into test1 (name,id,street,city,zip) values(%s,%s,%s,%s,%s)"
data = ('Tommy', 2, 'Test 2nd Ave.', 'Seattle', '12345')

cur.mogrify(query2, data)

b"insert into test1 (name,id,street,city,zip) values('Tommy',2,'Test 2nd Ave.','Seattle','12345')"

In [16]:
cur.execute(query2, data)

In [17]:
conn.commit()

In [20]:
# extract data
select_all = "select * from test1"
cur.execute(select_all)

for record in cur:
    print(record)

('Tommy', 2, 'Test 2nd Ave.', 'Seattle', '12345')


In [30]:
# Test multiple records insert
from faker import Faker

fake = Faker()
data = []

for i in range(2, 100):
    data.append((fake.name(),
                 i,
                 fake.street_address(),
                 fake.city(),
                 fake.zipcode()))

data = tuple(data)
print(data[0])

('Lisa Vang', 2, '2067 Shawna Burgs Suite 185', 'Caseystad', '63525')


In [32]:
query_multiple = "insert into test1 (name,id,street,city,zip) values(%s,%s,%s,%s,%s)"

cur.mogrify(query_multiple, data[0])

b"insert into test1 (name,id,street,city,zip) values('Lisa Vang',2,'2067 Shawna Burgs Suite 185','Caseystad','63525')"

In [35]:
# executemany() to handle multiple inserts
cur.executemany(query_multiple, data)
conn.commit()

In [36]:
cur.execute(select_all)

for record in cur:
    print(record)

('Tommy', 2, 'Test 2nd Ave.', 'Seattle', '12345')
('Lisa Vang', 2, '2067 Shawna Burgs Suite 185', 'Caseystad', '63525')
('Matthew Perez', 3, '38145 James Circles Suite 614', 'Johntown', '18553')
('Robert Johnson', 4, '970 Peters Circles Apt. 756', 'Miketown', '80176')
('Jane Cook', 5, '577 Marissa Street', 'South Nicholashaven', '04980')
('Michael Wilson', 6, '95900 Sandra Shores Suite 436', 'New Johnny', '19305')
('Jillian Johnson', 7, '0043 Mason Hill Suite 515', 'Mariabury', '31073')
('Billy Patrick', 8, '70433 Carter Light Apt. 798', 'North Krystal', '14980')
('Eric Martinez', 9, '94306 Hughes Glen', 'East Lisa', '23522')
('Erica Munoz', 10, '84829 Robert Oval', 'Robertchester', '40384')
('Laurie Benitez', 11, '93708 Todd Parkways Apt. 307', 'New Derek', '57892')
('Hayden Melendez', 12, '7414 Victoria Fort Suite 545', 'Richardville', '09572')
('Corey Walker', 13, '520 Calvin Village Suite 903', 'New Joshua', '23406')
('Nicholas Long', 14, '85639 Courtney Overpass Apt. 968', 'North 

In [38]:
temp = cur.fetchone()
print(temp)

None


In [39]:
# Read records into pandas dataframe
import pandas as pd

df = pd.read_sql("select * from test1", conn)

df.head()

/home/parallels/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,id,street,city,zip
0,Tommy,2,Test 2nd Ave.,Seattle,12345
1,Lisa Vang,2,2067 Shawna Burgs Suite 185,Caseystad,63525
2,Matthew Perez,3,38145 James Circles Suite 614,Johntown,18553
3,Robert Johnson,4,970 Peters Circles Apt. 756,Miketown,80176
4,Jane Cook,5,577 Marissa Street,South Nicholashaven,04980


# NoSQL (ElasticSearch)

In [49]:
import elasticsearch
elasticsearch.__version__

(7, 6, 0)

In [50]:
from elasticsearch import Elasticsearch
from faker import Faker
fake=Faker()

In [68]:
# Create a connection to Elasticsearch running on localhost
es = Elasticsearch('https://localhost:9200',
                   http_auth=('username', 'passoword'),
                    ca_certs='/home/parallels/local/elasticsearch-8.0.0/config/certs/http_ca.crt')

In [75]:
# Create a new index
es.indices.create(index="users")

In [82]:
# Create a new document in the index created above
data = {"name": fake.name(),"street": fake.street_address(),"city": fake.city(), "test":fake.zipcode()}
res = es.index(index="users", body=data)
print(res['result'])

# Ref. https://elasticsearch-py.readthedocs.io/en/7.x/#

created


In [91]:
# Insert multiple documents
from elasticsearch import helpers

documents = [{
    "_index": "users",
    "_source": {
        "name": fake.name(),
        "street": fake.street_address(),
        "city": fake.city(),
        "zip":fake.zipcode()}} for x in range(999)
]

res = helpers.bulk(es, documents)

print(res)

(999, [])


In [98]:
# Query documents
doc = {"query":{"match_all":{}}}
# Can also use 'match'
#doc = {"query":{"match":{"name":"xxx"}}}

res = es.search(index="users",body=doc,size=10)
# Or instead of passing doc, use 'q' parameter
#res = es.search(index="users", q="name:xxx", size=10)

for doc in res['hits']['hits']:
    print(doc['_source'])

{'name': 'Terry Williamson', 'street': '045 Gonzalez Knolls', 'city': 'Barronville', 'test': '05361'}
{'name': 'Mary Barnes', 'street': '344 Brenda Rapids', 'city': 'Williamland', 'test': '57448'}
{'name': 'Lauren Warren', 'street': '33563 Eric Manor Suite 923', 'city': 'Wendyhaven', 'zip': '45194'}
{'name': 'Brian Roberts', 'street': '1349 Davidson Views', 'city': 'Lake Rhondaport', 'zip': '55966'}
{'name': 'Jessica Stephenson', 'street': '929 Taylor Plaza', 'city': 'Caseburgh', 'zip': '72676'}
{'name': 'Brad Vincent', 'street': '39780 Butler Fort', 'city': 'New Darrell', 'zip': '53433'}
{'name': 'Brenda Brown', 'street': '69542 Kyle Shores', 'city': 'Lake Louis', 'zip': '80293'}
{'name': 'Molly Cox', 'street': '17901 Bush Trafficway Apt. 414', 'city': 'Fraziermouth', 'zip': '82641'}
{'name': 'Grant Murray', 'street': '216 Pollard Alley Suite 691', 'city': 'Port Linda', 'zip': '84085'}
{'name': 'Dr. Alexandra Castaneda', 'street': '1057 Sanders Fall Apt. 774', 'city': 'Lake Daniel', '

In [100]:
# Store the results to dataframe
import pandas as pd
df = pd.json_normalize(res['hits']['hits'])
df.head()

,_index,_id,_score,_source.name,_source.street,_source.city,_source.test,_source.zip
0,users,rwtGaH8BKokg3yJlfn8U,1.0,Terry Williamson,045 Gonzalez Knolls,Barronville,05361,NaN
1,users,sAtLaH8BKokg3yJlqH82,1.0,Mary Barnes,344 Brenda Rapids,Williamland,57448,NaN
2,users,sQtQaH8BKokg3yJlSn_R,1.0,Lauren Warren,33563 Eric Manor Suite 923,Wendyhaven,NaN,45194
3,users,sgtQaH8BKokg3yJlSn_R,1.0,Brian Roberts,1349 Davidson Views,Lake Rhondaport,NaN,55966
4,users,swtQaH8BKokg3yJlSn_R,1.0,Jessica Stephenson,929 Taylor Plaza,Caseburgh,NaN,72676


In [103]:
# Scroll results
res = es.search(
    index = 'users',
    scroll = '20m',
    size = 500,
    body = {"query":{"match_all":{}}}
)

sid = res['_scroll_id']
size = len(res['hits']['hits'])

while (size > 0):
    res = es.scroll(scroll_id = sid, scroll = '20m')
    
    sid = res['_scroll_id']
    size = len(res['hits']['hits'])
    
    for doc in res['hits']['hits']:
        print(doc['_source'])

{'name': 'Tara Fleming', 'street': '50690 Robert Plain Suite 710', 'city': 'Port Darrylburgh', 'zip': '95392'}
{'name': 'Laura Moran', 'street': '37206 Martinez Overpass', 'city': 'East Angela', 'zip': '86938'}
{'name': 'David Martin', 'street': '446 Vaughn Unions Apt. 235', 'city': 'Lisafort', 'zip': '68616'}
{'name': 'Harold Mills MD', 'street': '160 Sarah Pines Apt. 085', 'city': 'New Johnchester', 'zip': '36357'}
{'name': 'Donald Johnson', 'street': '504 Shaw Station', 'city': 'South Mariahshire', 'zip': '77302'}
{'name': 'Robert Johnson', 'street': '608 Brown Brooks Suite 932', 'city': 'Port Tiffany', 'zip': '40499'}
{'name': 'Timothy Olson', 'street': '0176 Dillon Shores', 'city': 'Caitlinberg', 'zip': '80129'}
{'name': 'Willie Campbell', 'street': '532 Julia Lodge', 'city': 'South Elaine', 'zip': '74352'}
{'name': 'Michael Wagner', 'street': '918 Green Walk Suite 679', 'city': 'Chavezport', 'zip': '52073'}
{'name': 'Colton Kim', 'street': '53189 Patrick Alley', 'city': 'Penaview